In [1]:
import ast
import cv2 as cv
import easyocr
from glob import glob
import numpy as np
import pandas as pd
import string
from ultralytics import YOLO

In [4]:
# regular pre-trained yolov8 model for car recognition
# coco_model = YOLO('yolov8n.pt')
model = YOLO('yolo11n.pt')
# yolov8 model trained to detect number plates
np_model = YOLO('license_plate_detection.pt')

FileNotFoundError: [Errno 2] No such file or directory: 'runs\\detect\\train4\\weights\\best.pt'